# 데이터 로드 및 사전처리
## 5. text 데이터 로드
#### 참고사이트 : https://www.tensorflow.org/tutorials/load_data/text?hl=ko

In [1]:
import tensorflow as tf

import tensorflow_datasets as tfds
import os

이번 예제에서 사용 할 3가지 번역본은 아래와 같음   
1. [Wiliam Cowper](https://en.wikipedia.org/wiki/William_Cowper)   
2. [Edward Smith Derby](https://en.wikipedia.org/wiki/Edward_Smith-Stanley,_14th_Earl_of_Derby)   
3. [Samuel Butler](https://en.wikipedia.org/wiki/Samuel_Butler_%28novelist%29)   

위 텍스트 파일들은 헤더, 바닥글, 줄 번호, 챕터 제목 등을 제거하는 전처리 작업을 거침.

In [2]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

'/home/kismi/.keras/datasets'

In [8]:
# dataset에 text data(file) 로드하기
# 각 예제에는 개별적으로 레이블을 지정해줘야 함.
# tf.data.Dataset.map 기능을 활용해 text data에 대해 (example, label) 쌍을 반환
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

In [10]:
# label이 지정된 dataset를 단일 셋으로 만들고 섞기(shuffle)
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [11]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)

In [13]:
for ex in all_labeled_data.take(5):
  print(ex)

# 출력 결과는 (example, label) 쌍이며 numpy 속성은 각 tensor의 값을 의미

(<tf.Tensor: id=391, shape=(), dtype=string, numpy=b'by the Trojans like an immortal, and the three sons of Antenor,'>, <tf.Tensor: id=392, shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: id=393, shape=(), dtype=string, numpy=b"WHEN by their sev'ral chiefs the troops were rang'd,">, <tf.Tensor: id=394, shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: id=395, shape=(), dtype=string, numpy=b'Encouraging, thus spake Antilochus.'>, <tf.Tensor: id=396, shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: id=397, shape=(), dtype=string, numpy=b"In stature less than Atreus' royal son,">, <tf.Tensor: id=398, shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: id=399, shape=(), dtype=string, numpy=b'Tell them, and the effect shall sure ensue,'>, <tf.Tensor: id=400, shape=(), dtype=int64, numpy=0>)


In [14]:
# text line을 숫자로 인코딩
# 기계학습모델은 단어가 아닌 숫자에 대해 작동하기 때문에 문자 -> 숫자 변환(고유한 단어를 고유한 숫자로 변환)

# 어휘 만들기
# text를 개별 단어 모음으로 토큰화하여 어휘로 변환
# 1) numpy 값을 반복
# 2) tfds.features.text.Tokenizer를 통해 split
# 3) 토큰을 python set으로 만들고 중복 제거
# 4) 나중에 사용 할 수 있도록 어휘의 size 측정
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

17178

In [16]:
# 인코딩
# text 문자열을 받아서 정수 목록을 반환
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)
print(encoder)

<TokenTextEncoder vocab_size=17180>


In [19]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'by the Trojans like an immortal, and the three sons of Antenor,'


In [20]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[9762, 4077, 4734, 9900, 1166, 12603, 212, 4077, 5157, 13187, 10487, 16898]


In [21]:
# tf.py_function dataset의 map 메소드에 text data를 전달해 encode 실행
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

In [22]:
# dataset.map 함수를 dataset 각 요소에 적용하고자 함.
# 하지만 그래프 모드에는 tensor에는 값이 없고 제한적인 기능만 사용 할 수 있기 때문에
# tf.py_function 함수를 통해 매핑해야 함.
def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label

all_encoded_data = all_labeled_data.map(encode_map_fn)

In [25]:
# dataset을 test 및 train_bach로 분할
# tf.data.Dataset.take 및 skip을 사용
# model에 전달되기 전에 dataset을 일괄 처리 해야함.
# 각 text line의 단어 수는 다른데 이걸 padded_batch 함수를 통해 동일한 크기로 맞추기
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([None],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([None],[]))

In [27]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

(<tf.Tensor: id=99918, shape=(16,), dtype=int64, numpy=
 array([ 9762,  4077,  4734,  9900,  1166, 12603,   212,  4077,  5157,
        13187, 10487, 16898,     0,     0,     0,     0])>,
 <tf.Tensor: id=99922, shape=(), dtype=int64, numpy=2>)

In [28]:
# 새로운 토큰 인코딩(패딩에 사용되는 0)을 도입했기 때문에 어휘 크기가 1씩 증가
vocab_size += 1

In [30]:
# 모델 구축
model = tf.keras.Sequential()

# 첫 번째 layer는 정수를 vector embedding으로 변환
model.add(tf.keras.layers.Embedding(vocab_size, 64))

# 두 번째 layer는 Long Short-Term Memory 계층(모델이 문맥에서 단어를 이해할 수 있도록 함)
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

In [32]:
# 위 과정을 통해 일련의 layer를 가지는 모델 가짐
# 출력 layer는 모든 레이블에 대한 확률을 생성
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3))

In [33]:
# 모델 compile(손실함수 지정 등)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [36]:
# 모델 훈련
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
      1/Unknown - 15s 15s/step

InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [8,3] and labels shape [1]
	 [[node loss/dense_5_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at /home/kismi/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_106291]

Function call stack:
distributed_function


In [37]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

79/79 [==============================] - 4s 52ms/step - loss: 1.0985 - accuracy: 0.3822

Eval loss: 1.099, Eval accuracy: 0.382
